In [1]:
from kv import KV

kv = KV.of('sql+sqlite:///dev.db?table=kv', type=str)

In [2]:
await kv.insert('a/b/c', 'hello')
await kv.insert('a/b/d', 'hello')
await kv.insert('a/c/d', 'hello')
await kv.insert('a/c/e', 'hello')
await kv.insert('b/c/e', 'hello')
await kv.insert('b/d/e', 'hello')

In [4]:
[k async for k in kv.prefix('a/c').keys()]

['a/c/d', 'a/c/e']